# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [4]:
# Create our features
# YOUR CODE HERE
X = df.copy()
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)
# Create our target
# YOUR CODE HERE
# y = df.loc[:,["loan_status"]]
# y = pd.get_dummies(df['loan_status'])
# y = pd.DataFrame(df["loan_status"])#.values.reshape(-1,1))
y = df["loan_status"]

In [5]:
X.describe()
# X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [6]:
# Check the balance of our target values
# y['loan_status'].value_counts()#- prefilled code returns KeyError
# y.value_counts()
Counter(y)

Counter({'low_risk': 75036, 'high_risk': 2500})

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
y = le.transform(y)
Counter(y)

Counter({1: 75036, 0: 2500})

In [8]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
                                                    #stratify=y)
# X_train.shape
# y_train


In [9]:
# print(f"Training target statistics: {Counter(y_train)}")
# print(f"Testing target statisictics: {Counter(y_test)}")

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(solver='lbfgs', random_state=1)
model_lr.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_lr = model_lr.predict(X_test_scaled)
lr_bas = balanced_accuracy_score(y_test, y_pred_lr)
lr_bas

0.9889115309798473

In [15]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
lr_cm = confusion_matrix(y_test, y_pred_lr)

lr_cm_df = pd.DataFrame(
    lr_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Display the confusion matrix
lr_cm_df

,Predicted 0,Predicted 1
Actual 0,609,10
Actual 1,113,18652


In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
lr_report = classification_report_imbalanced(y_test, y_pred_lr)
print(lr_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.98      0.99      0.91      0.99      0.98       619
          1       1.00      0.99      0.98      1.00      0.99      0.98     18765

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



In [17]:
# Recall Score
from imblearn.metrics import sensitivity_score
lr_recall = sensitivity_score(y_test, y_pred_lr)
print(lr_recall)

0.9939781508126831


In [18]:
# Geometric Mean Score
from imblearn.metrics import geometric_mean_score
lr_gms = geometric_mean_score(y_test, y_pred_lr)
print(lr_gms)

0.9888985516565261


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [19]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
print(f"Training target statistics: {Counter(y_resampled_ros)}")
print(f"Testing target statisictics: {Counter(y_test)}")

Training target statistics: Counter({1: 56271, 0: 56271})
Testing target statisictics: Counter({1: 18765, 0: 619})


In [20]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_ros = LogisticRegression(solver='lbfgs', random_state=1)
model_ros.fit(X_resampled_ros, y_resampled_ros)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_ros = model_ros.predict(X_test_scaled)
ros_bas = balanced_accuracy_score(y_test, y_pred_ros)
ros_bas

0.9934649587814939

In [22]:
# Display the confusion matrix
# YOUR CODE HERE
ros_cm = confusion_matrix(y_test, y_pred_ros)
ros_cm

array([[  615,     4],
       [  124, 18641]], dtype=int64)

In [23]:
# Print the imbalanced classification report
# YOUR CODE HERE
ros_report = classification_report_imbalanced(y_test, y_pred_ros)
print(ros_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.99      0.99      0.91      0.99      0.99       619
          1       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [24]:
# Recall Score
ros_recall = sensitivity_score(y_test, y_pred_ros)
print(ros_recall)

0.9933919531041833


In [25]:
# Geometric Mean Score
ros_gms = geometric_mean_score(y_test, y_pred_ros)
print(ros_gms)

0.9934649560990495


### SMOTE Oversampling

In [26]:
# Resample the training data with SMOTE
# YOUR CODE HERE
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=1, sampling_strategy=1.0)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
# YOUR CODE HERE
print(f"Training target statistics: {Counter(y_resampled_smote)}")
print(f"Testing target statisictics: {Counter(y_test)}")

Training target statistics: Counter({1: 56271, 0: 56271})
Testing target statisictics: Counter({1: 18765, 0: 619})


In [27]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_smote = LogisticRegression(solver='lbfgs', random_state=1, max_iter=1000)
model_smote.fit(X_resampled_smote, y_resampled_smote)

LogisticRegression(max_iter=1000, random_state=1)

In [28]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred_smote = model_smote.predict(X_test_scaled)
smote_bas = balanced_accuracy_score(y_test, y_pred_smote)
smote_bas

0.9934649587814939

In [29]:
# Display the confusion matrix
# YOUR CODE HERE
smote_cm = confusion_matrix(y_test, y_pred_smote)
smote_cm

array([[  615,     4],
       [  124, 18641]], dtype=int64)

In [30]:
# Print the imbalanced classification report
# YOUR CODE HERE
smote_report = classification_report_imbalanced(y_test, y_pred_smote)
print(smote_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.99      0.99      0.91      0.99      0.99       619
          1       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [31]:
# Recall Score
smote_recall = sensitivity_score(y_test, y_pred_smote)
print(smote_recall)

0.9933919531041833


In [32]:
# Geometric Mean Score
smote_gms = geometric_mean_score(y_test, y_pred_smote)
print(smote_gms)

0.9934649560990495


# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [33]:
# Resample the data using the ClusterCentroids resampler
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train_scaled, y_train)

In [34]:
# View the count of target classes with Counter
# YOUR CODE HERE
print(f"Training target statistics: {Counter(y_resampled_cc)}")
print(f"Testing target statisictics: {Counter(y_test)}")

Training target statistics: Counter({0: 1881, 1: 1881})
Testing target statisictics: Counter({1: 18765, 0: 619})


In [35]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

LogisticRegression(random_state=1)

In [36]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred_cc = model_cc.predict(X_test_scaled)
cc_bas = balanced_accuracy_score(y_test, y_pred_cc)
cc_bas

0.9929503031930944

In [37]:
# Display the confusion matrix
# YOUR CODE HERE
cc_cm = confusion_matrix(y_test, y_pred_cc)
cc_cm

array([[  614,     5],
       [  113, 18652]], dtype=int64)

In [38]:
# Print the imbalanced classification report
# YOUR CODE HERE
cc_report = classification_report_imbalanced(y_test, y_pred_cc)
print(cc_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.84      0.99      0.99      0.91      0.99      0.99       619
          1       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [39]:
# Recall Score
cc_recall = sensitivity_score(y_test, y_pred_cc)
print(cc_recall)

0.9939781508126831


In [40]:
# Geometric Mean Score
cc_gms = geometric_mean_score(y_test, y_pred_cc)
print(cc_gms)

0.9929497712072495


# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [41]:
# Resample the training data with SMOTEENN
# YOUR CODE HERE
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled_smoteenn, y_resampled_smoteenn = smote_enn.fit_resample(X_train_scaled, y_train)
# View the count of target classes with Counter
# YOUR CODE HERE
print(f"Training target statistics: {Counter(y_resampled_smoteenn)}")
print(f"Testing target statisictics: {Counter(y_test)}")

Training target statistics: Counter({1: 55934, 0: 55653})
Testing target statisictics: Counter({1: 18765, 0: 619})


In [42]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1, max_iter=1000)
model_smoteenn.fit(X_resampled_smoteenn, y_resampled_smoteenn)

LogisticRegression(max_iter=1000, random_state=1)

In [43]:
# Calculate the balanced accuracy score
# YOUR CODE HERE
y_pred_smoteenn = model_smoteenn.predict(X_test_scaled)
smoteenn_bas = balanced_accuracy_score(y_test, y_pred_smoteenn)
smoteenn_bas

0.9934649587814939

In [44]:
# Display the confusion matrix
# YOUR CODE HERE
smoteenn_cm = confusion_matrix(y_test, y_pred_smoteenn)
smoteenn_cm

array([[  615,     4],
       [  124, 18641]], dtype=int64)

In [45]:
# Print the imbalanced classification report
# YOUR CODE HERE
smoteenn_report = classification_report_imbalanced(y_test, y_pred_smoteenn)
print(smoteenn_report)

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.99      0.99      0.91      0.99      0.99       619
          1       1.00      0.99      0.99      1.00      0.99      0.99     18765

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



In [46]:
# Recall Score
smoteenn_recall = sensitivity_score(y_test, y_pred_smoteenn)
print(smoteenn_recall)

0.9933919531041833


In [47]:
# Geometric Mean Score
smoteenn_gms = geometric_mean_score(y_test, y_pred_smoteenn)
print(smoteenn_gms)

0.9934649560990495


# Final Questions

**1. Which model had the best balanced accuracy score?**

**ANSWER: According to table below ROS, SMOTE and SMOTEENN all had the equal best balanced accuracy score of 0.9934649587814939** 

In [48]:
print(f"Balanced Accuracy Scores Sampling Comparisons:")
print(f"----------------------------------------")
print(f"Logistic Regression Baseline:        {lr_bas}")
print(f"Random Over Sampling:                {ros_bas}")
print(f"SMOTE Over Sampling:                 {smote_bas}")
print(f"Cluster Centroids Under Sampling:    {cc_bas}")
print(f"SMOTEENN Combination Sampling:       {smoteenn_bas}")

Balanced Accuracy Scores Sampling Comparisons:
----------------------------------------
Logistic Regression Baseline:        0.9889115309798473
Random Over Sampling:                0.9934649587814939
SMOTE Over Sampling:                 0.9934649587814939
Cluster Centroids Under Sampling:    0.9929503031930944
SMOTEENN Combination Sampling:       0.9934649587814939


**2. Which model had the best recall score?**

**ANSWER: According to table below ClusterCentroids had the best recall score of 0.9939781508126831** 

In [49]:
print(f"Recall Scores Sampling Comparisons:")
print(f"----------------------------------------")
print(f"Logistic Regression Baseline:        {lr_recall}")
print(f"Random Over Sampling:                {ros_recall}")
print(f"SMOTE Over Sampling:                 {smote_recall}")
print(f"Cluster Centroids Under Sampling:    {cc_recall}")
print(f"SMOTEENN Combination Sampling:       {smoteenn_recall}")

Recall Scores Sampling Comparisons:
----------------------------------------
Logistic Regression Baseline:        0.9939781508126831
Random Over Sampling:                0.9933919531041833
SMOTE Over Sampling:                 0.9933919531041833
Cluster Centroids Under Sampling:    0.9939781508126831
SMOTEENN Combination Sampling:       0.9933919531041833


**3. Which model had the best geometric mean score?**

**ANSWER: According to table below ROS, SMOTE and SMOTEENN all had the equal best geometric mean score of 0.9934649560990495** 

In [50]:
print(f"Geometric Mean Scores Sampling Comparisons:")
print(f"----------------------------------------")
print(f"Logistic Regression Baseline:        {lr_gms}")
print(f"Random Over Sampling:                {ros_gms}")
print(f"SMOTE Over Sampling:                 {smote_gms}")
print(f"Cluster Centroids Under Sampling:    {cc_gms}")
print(f"SMOTEENN Combination Sampling:       {smoteenn_gms}")

Geometric Mean Scores Sampling Comparisons:
----------------------------------------
Logistic Regression Baseline:        0.9888985516565261
Random Over Sampling:                0.9934649560990495
SMOTE Over Sampling:                 0.9934649560990495
Cluster Centroids Under Sampling:    0.9929497712072495
SMOTEENN Combination Sampling:       0.9934649560990495
